In [1]:
using SNNT
using JLD
using Glob

Precompiling SNNT
  ✓ PyCall
  ✓ SNNT
  2 dependencies successfully precompiled in 21 seconds. 166 already precompiled.


In [2]:
using Plots

In [131]:
include("Analysis.jl")

plot_trajectory2 (generic function with 1 method)

# Vertical CAS Advisories

**Assumptions:**
- Ignore any COC

### Previous Advisory: DNC
Note, that this analysis is incomplete (i.e. was aborted before termination)

In [132]:
folder = "./vertcas-pra02-vI"
prefix = "vertcas-full-compressed-pra02-vI"

"vertcas-full-compressed-pra02-vI"

In [133]:
result_summary, metadata = summarize_and_load(folder,prefix)

Loading results from ./vertcas-pra02-vI/vertcas-full-compressed-pra02-vI-*.jld


(SNNT.VerifierInterface.OlnnvResult(SNNT.VerifierInterface.Unsafe, Any[SNNT.SMTInterface.SmtFilterMeta(15, 0), SNNT.SMTInterface.SmtFilterMeta(9, 0), SNNT.SMTInterface.SmtFilterMeta(15, 0), SNNT.SMTInterface.SmtFilterMeta(9, 0), SNNT.SMTInterface.SmtFilterMeta(40, 0), SNNT.SMTInterface.SmtFilterMeta(665, 1), SNNT.SMTInterface.SmtFilterMeta(1917, 3), SNNT.SMTInterface.SmtFilterMeta(9, 0), SNNT.SMTInterface.SmtFilterMeta(6, 0), SNNT.SMTInterface.SmtFilterMeta(2985, 4)  …  SNNT.SMTInterface.SmtFilterMeta(169, 0), SNNT.SMTInterface.SmtFilterMeta(502, 0), SNNT.SMTInterface.SmtFilterMeta(12, 0), SNNT.SMTInterface.SmtFilterMeta(9, 0), SNNT.SMTInterface.SmtFilterMeta(45, 0), SNNT.SMTInterface.SmtFilterMeta(227, 0), SNNT.SMTInterface.SmtFilterMeta(5, 0), SNNT.SMTInterface.SmtFilterMeta(5, 0), SNNT.SMTInterface.SmtFilterMeta(202, 0), SNNT.SMTInterface.SmtFilterMeta(2253, 4)], SNNT.VerifierInterface.Star[SNNT.VerifierInterface.Star(Float32[0.0 -0.4667842 0.46678418 0.75114906; 0.0 -0.70710677 0.7

In [134]:
println("Status:")
println(result_summary.status)
println("Enumerated Stars:")
println(sum(x->x.original_meta,result_summary.metadata))
println("Filtered Stars:")
println(sum(x->x.filtered_out,result_summary.metadata))

Status:
Unsafe
Enumerated Stars:
11397
Filtered Stars:
13


In [135]:
print("Certain counter-examples:")
print(length(filter(x->x.certain,result_summary.stars)))

Certain counter-examples:4149

In [136]:
print("Concrete crashes found using heuristic: ")
concrete_crashes = filter(x->acas_simulate(x;do_print=false)|>has_nmac,filter(x->x[6]!="COC",map(x->acas_interpret_cex(x.counter_example;do_print=false),filter(x->x.certain,result_summary.stars))))
#concrete_crashes = filter(x->acas_simulate(x;do_print=false)|>has_nmac,map(x->acas_interpret_cex(x.counter_example;do_print=false)),result_summary.stars)
print(length(concrete_crashes))

Concrete crashes found using heuristic: 2037

In [137]:
closest_nmac = argmin(x->acas_simulate(x;vI_override=0.0)|>minimum_distance,concrete_crashes)
cex_i = closest_nmac
#cex_i = concrete_crashes[closest_nmac_i]
#cex_i = concrete_crashes[12000]
println(cex_i)
crash_trace = acas_simulate(cex_i)
print("Has NMAC: ", has_nmac(crash_trace))

(914.00146f0, 0.012207031f0, -97.302246f0, 22915.15f0, 10.1887045f0, "CL1500")
Has NMAC: true

Counterfactual: Keep DNC course

In [138]:
cex_cf = [x for x in cex_i]
cex_cf[6]="DNC"
print(cex_cf)
cf_trace = acas_simulate(cex_cf)
print("Has NMAC: ", has_nmac(cf_trace))

Any[914.00146f0, 0.012207031f0, -97.302246f0, 22915.15f0, 10.1887045f0, "DNC"]Has NMAC: false

In [139]:
crash_trace[1]

(914.00146f0, 22915.15f0, 0.012207031f0, -97.302246f0)

In [140]:
crash_trace

152-element Vector{NTuple{4, Float32}}:
 (914.00146, 22915.15, 0.012207031, -97.302246)
 (904.27, 22695.15, 0.012207031, -97.302246)
 (894.5386, 22475.15, 0.012207031, -97.302246)
 (884.8071, 22255.15, 0.012207031, -97.302246)
 (875.0757, 22035.15, 0.012207031, -97.302246)
 (865.34424, 21815.15, 0.012207031, -97.302246)
 (855.6128, 21595.15, 0.012207031, -97.302246)
 (845.88135, 21375.15, 0.012207031, -97.302246)
 (836.1499, 21155.15, 0.012207031, -97.302246)
 (826.41846, 20935.15, 0.012207031, -97.302246)
 (816.687, 20715.15, 0.012207031, -97.302246)
 (806.95557, 20495.15, 0.012207031, -97.302246)
 (797.2241, 20275.15, 0.012207031, -97.302246)
 ⋮
 (-448.40088, -7884.8496, 0.012207031, -97.302246)
 (-458.13232, -8104.8496, 0.012207031, -97.302246)
 (-467.86377, -8324.85, 0.012207031, -97.302246)
 (-477.5952, -8544.85, 0.012207031, -97.302246)
 (-487.32666, -8764.85, 0.012207031, -97.302246)
 (-497.0581, -8984.85, 0.012207031, -97.302246)
 (-506.78955, -9204.85, 0.012207031, -97.302246)

In [141]:
include("Analysis.jl")
plot_trajectory2("DNC-vI-rerun",crash_trace,cf_trace,show_planes=true,xlims=(-11.2,11.2),ylims=(9400,11100),delta=3.0,plane_offset=0.7,thetai_offset=0.2,xishift=-0.0)

0.013362871370813087
3.0
0.00023322610294117645


"/home/samuel/Dokumente/Projects/CPS/NN-Safety-via-dL/repos/NCubeV-Reinstall/experiments/acas/DNC-vI-rerun.png"

### Previous Advisory: DND

In [142]:
folder = "./vertcas-pra03-vI"
prefix = "vertcas-full-compressed-pra03-vI"

"vertcas-full-compressed-pra03-vI"

In [143]:
result_summary, metadata = summarize_and_load(folder,prefix)

Loading results from ./vertcas-pra03-vI/vertcas-full-compressed-pra03-vI-*.jld


(SNNT.VerifierInterface.OlnnvResult(SNNT.VerifierInterface.Unsafe, Any[SNNT.SMTInterface.SmtFilterMeta(105, 0), SNNT.SMTInterface.SmtFilterMeta(3, 0), SNNT.SMTInterface.SmtFilterMeta(3, 0), SNNT.SMTInterface.SmtFilterMeta(7, 0), SNNT.SMTInterface.SmtFilterMeta(2, 0), SNNT.SMTInterface.SmtFilterMeta(4, 0), SNNT.SMTInterface.SmtFilterMeta(13, 0), SNNT.SMTInterface.SmtFilterMeta(13, 0), SNNT.SMTInterface.SmtFilterMeta(14, 0), SNNT.SMTInterface.SmtFilterMeta(16, 0)  …  SNNT.SMTInterface.SmtFilterMeta(6, 0), SNNT.SMTInterface.SmtFilterMeta(5, 0), SNNT.SMTInterface.SmtFilterMeta(203, 0), SNNT.SMTInterface.SmtFilterMeta(82, 0), SNNT.SMTInterface.SmtFilterMeta(10, 0), SNNT.SMTInterface.SmtFilterMeta(1116, 0), SNNT.SMTInterface.SmtFilterMeta(74, 0), SNNT.SMTInterface.SmtFilterMeta(34, 0), SNNT.SMTInterface.SmtFilterMeta(40, 0), SNNT.SMTInterface.SmtFilterMeta(3327, 0)], SNNT.VerifierInterface.Star[SNNT.VerifierInterface.Star(Float32[0.0 0.46678418 -0.4667842 0.75114906; 0.0 0.70710677 -0.707106

In [144]:
println("Status:")
println(result_summary.status)
println("Enumerated Stars:")
println(sum(x->x.original_meta,result_summary.metadata))
println("Filtered Stars:")
println(sum(x->x.filtered_out,result_summary.metadata))

Status:
Unsafe
Enumerated Stars:
6584
Filtered Stars:
0


In [145]:
print("Certain counter-examples:")
print(length(filter(x->x.certain,result_summary.stars)))

Certain counter-examples:69

In [146]:
print("Concrete crashes found using heuristic: ")
concrete_crashes = filter(x->acas_simulate(x;do_print=false)|>has_nmac,filter(x->x[6]!="COC",map(x->acas_interpret_cex(x.counter_example;do_print=false),filter(x->x.certain,result_summary.stars))))
#concrete_crashes = filter(x->acas_simulate(x;do_print=false)|>has_nmac,map(x->acas_interpret_cex(x.counter_example;do_print=false)),result_summary.stars)
print(length(concrete_crashes))

Concrete crashes found using heuristic: 37

In [147]:
closest_nmac = argmin(x->acas_simulate(x;vI_override=0.0)|>minimum_distance,concrete_crashes)
cex_i = closest_nmac
#cex_i = concrete_crashes[closest_nmac_i]
#cex_i = concrete_crashes[12000]
println(cex_i)
crash_trace = acas_simulate(cex_i)
print("Has NMAC: ", has_nmac(crash_trace))

(-1031.25f0, 1.5625f0, 98.4375f0, 25969.172f0, 11.576897f0, "DES1500")
Has NMAC: true

Counterfactual: Keep DND course

In [148]:
cex_cf = [x for x in cex_i]
cex_cf[6]="DND"
print(cex_cf)
cf_trace = acas_simulate(cex_cf)
print("Has NMAC: ", has_nmac(cf_trace))

Any[-1031.25f0, 1.5625f0, 98.4375f0, 25969.172f0, 11.576897f0, "DND"]Has NMAC: false

In [149]:
include("Analysis.jl")
plot_trajectory2("DND-vI-rerun",crash_trace,cf_trace,show_planes=true,xlims=(-13,13),ylims=(8800,10500),delta=4.5,plane_offset=0.7,thetai_offset=-0.3,xishift=0.7)

3.012241349401019
4.5
0.052573529411764706


"/home/samuel/Dokumente/Projects/CPS/NN-Safety-via-dL/repos/NCubeV-Reinstall/experiments/acas/DND-vI-rerun.png"

In [20]:
include("Analysis.jl")
plot_trajectory2("DND-vI-rerun",crash_trace,cf_trace;ylims=(9400,11000))

"/home/samuel/Dokumente/Projects/CPS/NN-Safety-via-dL/repos/NCubeV-Reinstall/experiments/acas/DND-vI-rerun.png"

In [52]:
maximum(x->abs(x[3]),crash_trace)

1.5625f0

In [53]:
maximum(x->abs(x[3]),cf_trace)

98.9191f0